## Part 3 Implementing & Improving AlexNet

Importing Required Libraries

In [20]:
import numpy as np
import pandas as pd
import pickle

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

import torchvision
from torchvision.io import read_image
from torchvision import transforms
from torch.utils.data import Dataset

import torch
from torch import nn
import torch.optim as optim
from torch.utils.data import DataLoader

import matplotlib.pyplot as plt

In [21]:
print(torch.cuda.is_available())
device = torch.device('cuda')
device

True


device(type='cuda')

Step 1:

Loading the Data

In [5]:
# dataset = []
# for i in range(1, 10001):
#     img = torchvision.io.read_image(f'C:/Users/RaghulKrish/Desktop/UB/Spring 23/ML/Assignment/A_2/cnn_dataset/dogs/{i}.jpg')
#     dataset.append(img)
# for i in range(1, 10001):
#     img = torchvision.io.read_image(f'C:/Users/RaghulKrish/Desktop/UB/Spring 23/ML/Assignment/A_2/cnn_dataset/food/{i}.jpg')
#     dataset.append(img)
# for i in range(1, 10001):
#     img = torchvision.io.read_image(f'C:/Users/RaghulKrish/Desktop/UB/Spring 23/ML/Assignment/A_2/cnn_dataset/vehicles/{i}.jpg')
#     dataset.append(img)

# labels = []
# for i in range(1, 10001):
#     labels.append(0)
# for i in range(1, 10001):
#     labels.append(1)
# for i in range(1, 10001):
#     labels.append(2)

In [13]:
data_path = "C:/Users/RaghulKrish/Desktop/UB/Spring 23/ML/Assignment/A_2/cnn_dataset/"
TRANSFORM_IMG = transforms.Compose([
    transforms.Resize(224),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225] )
    ])

data = torchvision.datasets.ImageFolder(root = data_path, transform = TRANSFORM_IMG)

In [27]:
# initialize empty arrays for tensors and labels
X = []
y = np.zeros(30000)

# loop through each item in the data and separate tensors and labels
for i, (tensor, label) in enumerate(data):
    X.append(tensor)
    y[i] = label

print(len(X))
print(len(y))

In [31]:
print(len(X))
print(len(y))

30000
30000


Step 2

In [5]:
# Alex Net
class AlexNet(nn.Module):
    def __init__(self, no_of_class = 3):
        super(AlexNet, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(3, 96, kernel_size = 11, stride = 4, padding = 0),
            nn.BatchNorm2d(96),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 3, stride = 2))
        self.layer2 = nn.Sequential(
            nn.Conv2d(96, 256, kernel_size = 5, stride = 1, padding = 2),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 3, stride = 2))
        self.layer3 = nn.Sequential(
            nn.Conv2d(256, 384, kernel_size = 3, stride = 1, padding = 1),
            nn.BatchNorm2d(384),
            nn.ReLU())
        self.layer4 = nn.Sequential(
            nn.Conv2d(384, 384, kernel_size = 3, stride = 1, padding = 1),
            nn.BatchNorm2d(384),
            nn.ReLU())
        self.layer5 = nn.Sequential(
            nn.Conv2d(384, 256, kernel_size = 3, stride = 1, padding = 1),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 3, stride = 2))
        self.fc1 = nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(9216, 4096),
            nn.ReLU())
        self.fc2 = nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(4096, 4096),
            nn.ReLU())
        self.fc3 = nn.Sequential(
            nn.Linear(4096, no_of_class))
    
    def forward(self, x):
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        x = self.layer5(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = self.fc2(x)
        x = self.fc3(x)
        return x